In [1]:
# miniconda3/ openai

import openai
import pandas as pd

In [2]:
# response = openai.Engine("ada").search(
#     search_model="ada", 
#     query="discrete stat distribution", 
#     max_rerank=3,
#     file="file-NdM8EkdPCKFmEcL9ksaKR27K",
#     return_metadata=True
# )
# response

<OpenAIObject list at 0x106011300> JSON: {
  "data": [
    {
      "document": 0,
      "metadata": {
        "date_created": null,
        "date_modified": null,
        "title": "(Alquier, 2021)\n"
      },
      "object": "search_result",
      "score": 159.076,
      "text": "(Alquier, 2021)\nAbstract: Aggregated predictors are obtained by making a set of basic predictors vote according to some weights, that is, to some probability distribution. Randomized predictors are obtained by sampling in a set of basic predictors, according to some prescribed probability distribution. Thus, aggregated and randomized predictors have in common that they are not defined by a minimization problem, but by a probability distribution on the set of predictors. In statistical learning theory, there is a set of tools designed to understand the generalization ability of such procedures: PAC-Bayesian or PAC-Bayes bounds. Since the original PAC-Bayes bounds of D. McAllester, these tools have been conside

In [7]:
for document in response['data']:

    print('Title: {}\n{}\n'.format(document['metadata']['title'], document['text']))

Title: (Alquier, 2021)

(Alquier, 2021)
Abstract: Aggregated predictors are obtained by making a set of basic predictors vote according to some weights, that is, to some probability distribution. Randomized predictors are obtained by sampling in a set of basic predictors, according to some prescribed probability distribution. Thus, aggregated and randomized predictors have in common that they are not defined by a minimization problem, but by a probability distribution on the set of predictors. In statistical learning theory, there is a set of tools designed to understand the generalization ability of such procedures: PAC-Bayesian or PAC-Bayes bounds. Since the original PAC-Bayes bounds of D. McAllester, these tools have been considerably improved in many directions (we will for example describe a simplified version of the localization technique of O. Catoni that was missed by the community, and later rediscovered as "mutual information bounds"). Very recently, PAC-Bayes bounds received

In [44]:
import json
hi = json.load(open("one_search_response.json"))
hi

{'object': 'list',
 'data': [{'object': 'search_result',
   'document': 0,
   'score': 158.83,
   'text': '(Alquier, 2021)\nAbstract: Aggregated predictors are obtained by making a set of basic predictors vote according to some weights, that is, to some probability distribution. Randomized predictors are obtained by sampling in a set of basic predictors, according to some prescribed probability distribution. Thus, aggregated and randomized predictors have in common that they are not defined by a minimization problem, but by a probability distribution on the set of predictors. In statistical learning theory, there is a set of tools designed to understand the generalization ability of such procedures: PAC-Bayesian or PAC-Bayes bounds. Since the original PAC-Bayes bounds of D. McAllester, these tools have been considerably improved in many directions (we will for example describe a simplified version of the localization technique of O. Catoni that was missed by the community, and later re

In [3]:
openai.api_key = None #YOUR API KEY HERE
openai.Engine.list()

<OpenAIObject list at 0x106ffffb0> JSON: {
  "data": [
    {
      "created": null,
      "id": "ada",
      "max_replicas": null,
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true,
      "replicas": null
    },
    {
      "created": null,
      "id": "ada-code-search-code",
      "max_replicas": null,
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true,
      "replicas": null
    },
    {
      "created": null,
      "id": "ada-code-search-text",
      "max_replicas": null,
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true,
      "replicas": null
    },
    {
      "created": null,
      "id": "ada-instruct-beta",
      "max_replicas": null,
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true,
      "replicas": null
    },
    {
      "created": null,
      "id": "ada-search-document",
      "max_re

In [27]:
# glob all md files in current directory into list of files
from pathlib import Path
# current working directory
# Path()
notion_export_path = Path.cwd() / r'Notion-exports'
mdfiles = list(notion_export_path.glob("**/*.md"))
print("found files:", len(mdfiles))

found files: 1056


# limits
https://beta.openai.com/docs/api-reference/searches

200 docs max per upload
docs cant be too long... 
The maximum document length (in tokens) is 2034 minus the number of tokens in the query.



The similarity score is a positive score that usually ranges from 0 to 300 (but can sometimes go higher), where a score above 200 usually means the document is semantically similar to the query.

In [29]:
# CLEAN NOTION NOTION DATABASE EXPORT (from right click, export as csv) 

data = []

for filepath in mdfiles:
    file = open(filepath, 'r')
    text = ""
    date_created = ""
    date_modified = ""
    for idx, line in enumerate(file.readlines()):
        if idx == 0:
            line = line.strip("# ")
            title = line
            # keep title in body
            # continue 
        while line.startswith(' '):
            line = line[1:]
        if line.startswith('Date Created:'):
            line = line[14:] # remove first 14 chars.
            date_created = line
            continue
        if line.startswith('Date Modified:'):
            line = line[15:] # remove first 15 chars for "Date Modified: "
            date_modified = line
            continue
        if line.startswith('[http'): 
            # Keeps links that have an alias!! Yay. Should remove link http part... and clean formatting
            # todo, keep just the [] part and not the () part in "[text](link)"
            continue
        if not line.startswith('!') and len(line) >= 3:
            text = text + str(line)
            
    # DANGER: limit each doc size!
    # also, only 200 doc limit. RN I have 135 docs.
    text = text[0:2500]
    data.append([title,date_created,date_modified, text])
    # print(text) 
    # break
    
df = pd.DataFrame(data=data)
df.columns = ["title","date_created","date_modified", "text"]
# save dataframe df as csv file
df.to_csv("firsttry_all_base_notes.csv", index=False)
df

,title,date_created,date_modified,text
0,Readwise\n,,,Readwise\n[Daily Reviews](Readwise%20db63a/Dai...
1,Curiosity\n,,,Curiosity\n[Curiosity Notes](Curiosity%200990c...
2,People\n,,,People\n[People database](People%20278e2/Peopl...
3,Things to Blog About\n,,,Things to Blog About\n[Revamping my blog](Thin...
4,Cuts from Designing Better Books\n,,,Cuts from Designing Better Books\nCuts from De...
...,...,...,...,...
1051,(Eykholt_Robust_Physical-World_Attacks_CVPR_20...,,,(Eykholt_Robust_Physical-World_Attacks_CVPR_20...
1052,"(Dean et al., n.d.)\n",,,"(Dean et al., n.d.)\nAbstract: We consider thr..."
1053,"(Sławek & Arodź, 2013)\n",,,"(Sławek & Arodź, 2013)\nAbstract: Background: ..."
1054,"(Dibaeinia & Sinha, 2020)\n",,,"(Dibaeinia & Sinha, 2020)\nAbstract: A common ..."


In [4]:
# pandas load csv to dataframe
notes = pd.read_csv("curiosity_notes_clean.csv")#,header=None)
notes

,title,date_created,date_modified,text
0,Plastics\n,"March 23, 2021 1:33 PM\n","March 23, 2021 1:42 PM\n",Plastics\n1 and 2 do get recycled. 7 is catch ...
1,Networking HTTP/3\n,"January 7, 2022 7:42 PM\n","January 7, 2022 8:19 PM\n",Networking HTTP/3\nHTTPS = TCP + TLS + HTTP\n-...
2,Date ideas\n,"December 22, 2020 3:28 PM\n","May 22, 2021 5:18 PM\n",Date ideas\nMuseums are looking nice \nthat ne...
3,energy to the moon\n,"March 7, 2021 4:53 PM\n","March 7, 2021 4:59 PM\n",energy to the moon\nname: energy —- in the props
4,engineering for change\n,"December 2, 2020 7:54 PM\n","March 14, 2021 4:46 PM\n",engineering for change\n# Sannitation\n[Sanita...
...,...,...,...,...
130,Second Call for Proposals Digital Transformati...,NaN,NaN,Second Call for Proposals Digital Transformati...
131,Quantum Fourier Transform\n,NaN,NaN,Quantum Fourier Transform\n[Untitled](Quantum%...
132,Quantum Eraser (Delayed Choice)\n,NaN,NaN,Quantum Eraser (Delayed Choice)\nCreated: Octo...
133,"When it comes down to it, the most obvious app...",NaN,NaN,"When it comes down to it, the most obvious app..."


In [5]:
# openai.File.create(file=open("myfile.jsonl"), purpose="search")

In [6]:
# {"text": "text of file"}

# filename = 'alltext.jsonl'

# for line in notes.text:
#     print(line)
#     out = 
#     out = f"\{\"text\"\}:\"{line}\""
#     break

In [30]:
# find total words
notes = pd.read_csv("firsttry_all_base_notes.csv")#,header=None)
alltext = notes.text.sum()

res = len(alltext.split())
res 

# 16_613 words for curiosity notes database
# 224_924 words for all base directory notes (counting space-delimited words)


224924

## Write Jsonl for OpenAI

In [32]:
# Encode csv into jsonl format for Openai File upload

notes = pd.read_csv("firsttry_all_base_notes.csv")#,header=None)

# create metatdata formated for Openai
notes['metadata'] = notes.apply(lambda row: {'date_created':row['date_created'], 'date_modified':row['date_modified'], 'title': row['title']}, axis=1)

notes.drop(["date_created", "date_modified", "title"], axis=1, inplace=True)

# notes.to_json(orient='records', lines=True)
# notes['merged']

# export to file
notes.to_json("allBaseNots_withMetadata.jsonl", orient='records', lines=True)

## Run search

In [34]:
openai.File.create(file=open("allBaseNots_withMetadata.jsonl"), purpose="search")

# <File file id=file-YvfHj7utJala3er24T0ZqeLb at 0x15eccf3d0> JSON: {
#   "bytes": 137010,
#   "created_at": 1649388503,
#   "filename": "curiosityNotes.jsonl",
#   "id": "file-YvfHj7utJala3er24T0ZqeLb",
#   "object": "file",
#   "purpose": "search",
#   "status": "uploaded",
#   "status_details": null
# }
# file-YvfHj7utJala3er24T0ZqeLb  # before metadata
# file-8wiFt4uSxNPRpaEwsCG0s6qD  # after including metadata
# file-NdM8EkdPCKFmEcL9ksaKR27K allBaseNots_withMetadata.jsonl

<File file id=file-NdM8EkdPCKFmEcL9ksaKR27K at 0x11e79cdb0> JSON: {
  "bytes": 1794603,
  "created_at": 1649561114,
  "filename": "allBaseNots_withMetadata.jsonl",
  "id": "file-NdM8EkdPCKFmEcL9ksaKR27K",
  "object": "file",
  "purpose": "search",
  "status": "uploaded",
  "status_details": null
}

In [36]:
response = openai.Engine("ada").search(
    search_model="ada", 
    query="discrete stat distribution", 
    max_rerank=7,
    file="file-NdM8EkdPCKFmEcL9ksaKR27K",
    return_metadata=True
)

<OpenAIObject list at 0x1283d75b0> JSON: {
  "data": [
    {
      "document": 0,
      "metadata": {
        "date_created": null,
        "date_modified": null,
        "title": "(Alquier, 2021)\n"
      },
      "object": "search_result",
      "score": 159.076,
      "text": "(Alquier, 2021)\nAbstract: Aggregated predictors are obtained by making a set of basic predictors vote according to some weights, that is, to some probability distribution. Randomized predictors are obtained by sampling in a set of basic predictors, according to some prescribed probability distribution. Thus, aggregated and randomized predictors have in common that they are not defined by a minimization problem, but by a probability distribution on the set of predictors. In statistical learning theory, there is a set of tools designed to understand the generalization ability of such procedures: PAC-Bayesian or PAC-Bayes bounds. Since the original PAC-Bayes bounds of D. McAllester, these tools have been conside

In [93]:
openai.Engine("ada").search(
    search_model="ada", 
    query="quantum", 
    max_rerank=5,
    file="file-YvfHj7utJala3er24T0ZqeLb"
)

<OpenAIObject list at 0x16bc9fd30> JSON: {
  "data": [
    {
      "document": 0,
      "object": "search_result",
      "score": 556.838,
      "text": "Quantum Mechanics (Big topic)\n[Andy Matuschak (Tools for Thought)](https://www.notion.so/Andy-Matuschak-Tools-for-Thought-7a44975cec684aa6a797df2e852bc79d) \nNeumonic medium\n[Topics](Quantum%20Me%205294f/Topics%20893cc.csv)\n# Best reading materials out there!!!\nQuanta Magazine \u2014 great mag, great videos. \nWolframphysics.org\n\u2b50 [David Deutch's Video Series Introducing Quantum Computation!!!!!](http://www.quiprocone.org/Protected/DD_lectures.htm)\n- WoW these videos are more than just math. They're theory and beauty of the multiverse.\n- [All his other resources](https://www.daviddeutsch.org.uk/videos/) (nice!)\nDifferent rules of probability from what we\u2019re used to. \nMath is the distilled heart of thinking. Chains of logic useful for anything. \nFrom the Map of Physics Youtuber. \n\u2b50\u2b50 [https://indico.cern.c

In [95]:
openai.Engine("ada").search(
    search_model="ada", 
    query="distributed computing", 
    max_rerank=5,
    file="file-YvfHj7utJala3er24T0ZqeLb"
)

<OpenAIObject list at 0x16ba5e1b0> JSON: {
  "data": [
    {
      "document": 0,
      "object": "search_result",
      "score": 198.224,
      "text": "AI Hackathon (Argon National Labs)\nReach out for Compute resources login, remote access.\nTom Gibbs Nvidia\n```python\nssh kastan@kastan@theta.alcf.anl.gov\npass: login number off of Pass+ iphone app. \n```\nSlack: [AI_Hackathon](https://app.slack.com/client/T02SPS0932P/C02S9FRCAR5) \nProj description: [AI_Hackathon_Molecular_Dynamics.pdf - Google Drive](https://drive.google.com/file/d/1ipGkyj2O1cE_XFp5lWIoY8JpFCrPmLld/view)\nMaking teams: [Teams_Hackathon - Google Docs](https://docs.google.com/document/d/12V1oq3kakSCmjAdyEHSZdwQq1qhDVNfCadiGHYRLaTo/edit)\nGPU docs: [Getting Started on ThetaGPU | Argonne Leadership Computing Facility (anl.gov)](https://www.alcf.anl.gov/support-center/theta-gpu-nodes/getting-started-thetagpu)\nTheta GPU Node trainings:  [Theta GPU Nodes | Argonne Leadership Computing Facility (anl.gov)](https://www.al

In [99]:
openai.Engine("ada").search(
    search_model="ada", 
    query="advanced computing", 
    max_rerank=5,
    file="file-YvfHj7utJala3er24T0ZqeLb"
)

<OpenAIObject list at 0x15e6cc450> JSON: {
  "data": [
    {
      "document": 0,
      "object": "search_result",
      "score": 152.89,
      "text": "webinar traning\nAll webinar recordings: [ALCF AI for Science Training Series | Argonne Leadership Computing Facility (anl.gov)](https://www.alcf.anl.gov/alcf-ai-science-training-series)\n# ENV\nPyTorch:\n- [ ]  \u2b50\u2b50\u00a0TORCHVISION OBJECT DETECTION FINETUNING TUTORIAL [https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html#torchvision-object-detection-finetuning-tutorial](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html#torchvision-object-detection-finetuning-tutorial)\nTensorflow:\nObject Detection API with Tensorflow 2 [https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2.md](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2.md)\nKeras: (detailed, code it from bottom up) \nObject Detection with RetinaNet [https://keras.io/ex

In [100]:
openai.Engine("ada").search(
    search_model="ada", 
    query="excellent researcher", 
    max_rerank=5,
    file="file-YvfHj7utJala3er24T0ZqeLb"
)

<OpenAIObject list at 0x16bc04860> JSON: {
  "data": [
    {
      "document": 0,
      "object": "search_result",
      "score": 280.174,
      "text": "iPad Apps\nInteresting iPad apps for creative notes \n# Reading academic papers\nReadcube Papers \u2014 seems lit for cross platform (all devices)... But no handwriting.\nGoodReader \u2014 powerful, cloud PDF editing. Has Handriting.\niOS only.\n\u2b50\u2b50\u2b50 **PaperShip** \u2b50\u2b50 \u2014 Hard to setup, but synch beautfiully with Zotero (or mendeley??) Excellent annotation. Replaced Goodreader for this person. \n- HANDWRITING SYNCH WITH MENDENELY OR ZOTERO!! IT'S PERFECT!!!\n## Ideal Workflow\nZotero (+ Zotfile) && annotations are extracted into plaintext!\nOr \u2665\u2665 **Papership** \u2665\u2665 ****+ Zotero or Mendenely \nOr Zotero desktop + zotero ipad app and that's it.\nOr Zotero + Zotfile + any pdf editor (including handwriting). Cuz the PDF file is synched via (any) cloud.\nTODO: Zotfile on Zotero. Advanced PDF mana

In [101]:
## GOOD ONE!!

openai.Engine("ada").search(
    search_model="ada", 
    query="good learning material", 
    max_rerank=6,
    file="file-YvfHj7utJala3er24T0ZqeLb"
)

<OpenAIObject list at 0x16bc9fe20> JSON: {
  "data": [
    {
      "document": 0,
      "object": "search_result",
      "score": 192.96,
      "text": "Curiosity Notes TransformerNet Comparison\nStudents struggle to pick out salient points (from new material) - must make required learning explicit.\nhttps://www.notion.so/8ebc461341ea4d9fb79f2994778625fe#4736170261c44ff483342443409a7ab8 \nParent: duplicate of Designing Better Books\nStudents struggle to pick out salient points - must make required learning explicit. \nhttps://www.notion.so/b4da244dc6fb487c8ec6b6f64873374e#9b82eae295ee42ffb2c6677ee8f6c43b \nParent: Meta-Learning\nScore: 0.93\nIndexes:  194 426 \n\u2023 \nhttps://www.notion.so/b4da244dc6fb487c8ec6b6f64873374e#3707d864b49d4da883b3c60789d267a4\nParent: Meta-Learning\n\u2023 \u2023 \nhttps://www.notion.so/b6355cce267e4d3dbc414af9e2449340#289d26ebf77142759575e1fe3f256e04\nParent: Media for thinking the unthinkable \nScore: 0.87\nIndexes:  395 502 \nBerkeley Graduate School T

In [ ]:
openai.Engine("ada").search(
    search_model="ada", 
    query="good learning material", 
    max_rerank=6,
    file="file-YvfHj7utJala3er24T0ZqeLb"
)

In [17]:
openai.Engine("ada").search(
    search_model="davinci", 
    query="good learning material", 
    max_rerank=6,
    file="file-YvfHj7utJala3er24T0ZqeLb"
)

<OpenAIObject list at 0x10f379120> JSON: {
  "data": [
    {
      "document": 0,
      "object": "search_result",
      "score": 192.96,
      "text": "Curiosity Notes TransformerNet Comparison\nStudents struggle to pick out salient points (from new material) - must make required learning explicit.\nhttps://www.notion.so/8ebc461341ea4d9fb79f2994778625fe#4736170261c44ff483342443409a7ab8 \nParent: duplicate of Designing Better Books\nStudents struggle to pick out salient points - must make required learning explicit. \nhttps://www.notion.so/b4da244dc6fb487c8ec6b6f64873374e#9b82eae295ee42ffb2c6677ee8f6c43b \nParent: Meta-Learning\nScore: 0.93\nIndexes:  194 426 \n\u2023 \nhttps://www.notion.so/b4da244dc6fb487c8ec6b6f64873374e#3707d864b49d4da883b3c60789d267a4\nParent: Meta-Learning\n\u2023 \u2023 \nhttps://www.notion.so/b6355cce267e4d3dbc414af9e2449340#289d26ebf77142759575e1fe3f256e04\nParent: Media for thinking the unthinkable \nScore: 0.87\nIndexes:  395 502 \nBerkeley Graduate School T

In [13]:
import os
# import openai
# openai.api_key = os.getenv("OPENAI_API_KEY")
openai.Completion.create(
  engine="text-davinci-002",
  prompt="A winning solution",
  max_tokens=40,
  temperature=1.5, # higher = more creative. Deafult: 1.
  top_p=0.3, #lower = limiting
)


<OpenAIObject text_completion id=cmpl-4vGlbfU6IukQEYMqSe8a8WiIekSMf at 0x10f3ef010> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " to the problem of a new approach to business\n\nThe problem of a new approach to business can be solved by adopting a customer-centric approach. This involves putting the customer at the center of everything"
    }
  ],
  "created": 1649554495,
  "id": "cmpl-4vGlbfU6IukQEYMqSe8a8WiIekSMf",
  "model": "text-davinci:002",
  "object": "text_completion"
}

In [18]:
df

NameError: name 'df' is not defined